In [323]:
Identifier_and_Name = {
    "Au-PeEL": "Ebook Library",
    "CaONFJC": "MyiLibrary",
    "CaPaEBR": "Ebrary",
    "DE-He213": "Springer-Verlag",
    "MiAaPQ": "ProQuest",
    "NhCcYBP": "YBP",
    "OCoLC": "OCLC",
    "SFPDA_MiAaPQ": "ProQuest",
    "YBPUID": "YBP",
    "MIL": "MyiLibrary",
    "EBR": "Ebrary",
    "EBL": "Ebook Library",
    "EBC": "Ebook Central",
    "CaBNVSL": "SIAM",
    "CaBNvSL": "SIAM",
    "DcWaAPA": "APA",
    "DNLM": "US Med Library",
    "FR-PaOEC": "OECD",
    "IN-ChSCO": "Scope e-Knowedge Center",
    "MiFhGG": "Gale",
    "StDuBDS": "Bibliographic Data Services",
    "UtOrBLW": "Backstage Library Works",
    "VaAlASP": "Alexander Street Press",
    "EBS": "EBSCO",
    "FANhCcYBP": "YBP",
}

> Start by loading Alpeh Sequential text file into OpenRefine with column widths 10,6,2

Put the unique name of the OpenRefine project for the BIB records in the variable `BIB_Project_Name` and run the cell

In [324]:
BIB_Project_Name = "oho_bib_3 18 2021 txt"

# Organize BIB Record

## Organize MARC Fields and Subfields
Copy Organize_Alpeh_Sequential.json and paste into OpenRefine

## Pivot Subfields into Columns
Use a filter on "No Field Number" to select the following subfields, invert, then remove all matching rows:

- 020$a
- 020$q
- 020$z
- 024$2
- 024$a
- 035$a
- 245$a
- 245$b
- 245$c
- 245$n
- 250$a
- 264$b
- 264$c
- 710$a
- 710$e
- 776$i
- 776$z
- 856$3
- 856$u
- 856$z
- 897$a
- 897$e

Open a text filter for "Fields," click on the number of choices to get the facet choices as TSV, paste into variable "Columns" below, and run cell

In [325]:
Columns = """
020$a	524
020$q	9
035$a	524
245$a	524
245$b	4
245$c	523
245$n	5
250$a	19
264$b	522
264$c	522
776$i	524
776$z	524
856$3	375
856$u	523
Field #1 264$b	2
Field #1 264$c	2
Field #2 264$c	2
"""

**Indicator columns deleted at this point--unsure how to handle/how or if to preserve beyond this point**

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [326]:
Columns = Columns.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Column_Names = []
for Column in Columns:
    Column = Column.rstrip("1234567890")
    Column = Column.rstrip("\t")
    Column_Names.append(Column)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for Column in Column_Names:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column}",
                "expression": "grel:row.record.cells[\\"{Column}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Column}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column}",
                "description": "Blank down cells in column ``{Column}``"
            }},
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

**BELOW ONLY WORKS IF NUMBER OF FIELDS IS LESS THAN 10; NO CHECK FOR COLLUMNS "246$b", **

In [327]:
def Column_Order_as_Letter(MARC_Tag):
    if MARC_Tag == "020$a":
        return "A"
    elif MARC_Tag == "020$z":
        return "B"
    elif MARC_Tag == "776$z":
        return "C"
    elif MARC_Tag == "020$q":
        return "D"
    elif MARC_Tag == "776$i":
        return "E"
    elif MARC_Tag == "024$a":
        return "F"
    elif MARC_Tag == "024$2":
        return "G"
    elif MARC_Tag == "035$a":
        return "H"
    elif MARC_Tag == "710$a":
        return "I"
    elif MARC_Tag == "710$e":
        return "J"
    elif MARC_Tag == "897$a":
        return "K"
    elif MARC_Tag == "897$e":
        return "L"
    elif MARC_Tag == "856$z":
        return "M"
    elif MARC_Tag == "856$3":
        return "N"
    elif MARC_Tag == "856$u":
        return "O"
    elif MARC_Tag == "264$c":
        return "P"
    elif MARC_Tag == "264$b":
        return "Q"
    elif MARC_Tag == "245$c":
        return "R"
    elif MARC_Tag == "250$a":
        return "S"
    elif MARC_Tag == "245$n":
        return "T"
    elif MARC_Tag == "245$b":
        return "U"
    elif MARC_Tag == "245$a":
        return "V"
    # No "else" statement because if there's a problem here, it will require manual intervention   

Column_Order = []
# Sort puts numbers before letters
for Column in Column_Names:
    if " " in Column: # Ordering_Letter, field number, underscore, MARC tags
        Field_Number = Column.split(" ")[1][1:]
        Ordering_Letter = Column_Order_as_Letter(Column.split(" ")[2])
        Column_Order.append(Ordering_Letter.lower() + Field_Number + "_" + Column)
    else: # Ordering_Letter, underscore, MARC tags
        Ordering_Letter = Column_Order_as_Letter(Column)
        Column_Order.append(Ordering_Letter + "1_" + Column)
Column_Order.sort()
Column_Order.sort(key=str.upper)
# The first sort sorts the elements without and with field numbers seperately; the second sort combines the two

Column_Order_Values = []
for Column in Column_Order:
    Column = Column.split("_")[1]
    Column_Order_Values.append('"' + Column + '"')

Reordering_String = ",\n".join(Column_Order_Values)
Template_String = f"""
      {{
    "op": "core/column-reorder",
    "columnNames": [
      "SYS Number",
      "Count",
      {Reordering_String}
    ],
    "description": "Reorder columns"
  }}
"""
print(Template_String)

        



      {
    "op": "core/column-reorder",
    "columnNames": [
      "SYS Number",
      "Count",
      "020$a",
"776$z",
"020$q",
"776$i",
"035$a",
"856$3",
"856$u",
"264$c",
"Field #1 264$c",
"Field #2 264$c",
"264$b",
"Field #1 264$b",
"245$c",
"250$a",
"245$n",
"245$b",
"245$a"
    ],
    "description": "Reorder columns"
  }



Run all the cells, copy the JSON that appears above, and paste it into OpenRefine

Copy Organize_Rarely_Duplicated_Fields.json and paste into OpenRefine

## Organize Publisher and Pub Date Fields

**PUBLISHER DUPLICATE FIELDS RELATED TO DUPLICATION OF INFORMATION NOT NEEDED, SO PIVOT LEFT NO RECORDS WITH MULTIPLE ROWS; WHEN THAT DOES HAPPEN, JSON WILL NEED TO BE ADJUSTED**

Copy Organize_Publisher_Fields.json and paste into OpenRefine

**PUB DATE DUPLICATE FIELDS IDENTFIED BY FILTERING FOR RECORDS WHERE "SYS NUMBER" HAD BLANK CELLS; FIRAT CASE HAD SAME DATE TWICE, SO SECOND DATE WSN'T PRESERVED--NEED TO FIGURE OUT STRATEGY FOR IF WE WANT TO PRESERVE BOTH DATES**

Copy Organize_Pub_Date_Fields.json and paste into OpenRefine

## Organize URLs

**URLS DIDN'T HAVE MULTIPLE FIELDS OR 856$Z TAG: NEED TO WORK OUT HOW TO USE WITH EITHER**

Copy Organize_URL_Fields_1.json and paste into OpenRefine

Open a text filter for "All 856$u 1", perform any clustering if appropriate, click on the number of choices to get the facet choices as TSV, paste into variable "URL_Headers" below, and run cell


In [328]:
URL_Headers = """
856$u	148
Oxford Handbooks Online	374
Oxford Scholarship Online	1
"""

Copy below and paste into OpenRefine

`{`

`"op": "core/key-value-columnize",`

`"keyColumnName": "All 856$u 1",`

`"valueColumnName": "All 856$u 2",`

`"noteColumnName": "",`
    
`"description": "Pivot data so values in ``All 856$u 1`` are columns with values from ``All 856$u 2``"`

`}`

Set "i" to the column number of column "020$a"

In [329]:
URL_Headers = URL_Headers.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Header_Names = []
for Header in URL_Headers:
    Header = Header.rstrip("1234567890")
    Header = Header.rstrip("\t")
    Header_Names.append(Header)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 9
    for Header in Header_Names:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Header}",
                "expression": "grel:row.record.cells[\\"{Header}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Header}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Header}",
                "description": "Blank down cells in column ``{Header}``"
            }},
            {{
                "op": "core/column-move",
                "columnName": "{Header}",
                "index": {i},
                "description": "Move column ``{Header}`` to position {i}"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

## Organize Identifier Fields

**Organize_Identifier_Fields HAS OTHER JSON STEPS FOR HANDLING NONSTANDARD FORMATTED 035$A FIELDS**

Copy Organize_Identifier_Fields.json and paste into OpenRefine

Open a text filter for "035$a 1", click on the number of choices to get the facet choices as TSV, paste into variable "Identifier_List" below, and run cell

In [330]:
Identifier_List = """
StDuBDS	524
"""

Copy below and paste into OpenRefine

`{`

`"op": "core/key-value-columnize",`

`"keyColumnName": "035$a 1",`

`"valueColumnName": "035$a 2",`

`"noteColumnName": "",`
    
`"description": "Pivot data so values in ``035$a 1`` are columns with values from ``035$a 2``"`

`}`

In [331]:
Identifier_List = Identifier_List.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Identifiers = []
for Identifier in Identifier_List:
    Identifier = Identifier.rstrip("1234567890")
    Identifier = Identifier.rstrip("\t")
    Identifiers.append(Identifier)

Old_and_New_Headers = dict()
for Identifier in Identifiers:
    if Identifier_and_Name.get(Identifier) is None:
        print(f"{Identifier} doesn't have a value. Add one to the dictionary Identifier_and_Name at the top of hte notebook and rerun all cells.")
        continue
    Old_and_New_Headers[Identifier] = Identifier_and_Name.get(Identifier)


Set "i" to the column number of column "Author"

In [332]:

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 7
    for Key, Value in Old_and_New_Headers.items():
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                "facets": [],
                "mode": "row-based"
                }},
                "baseColumnName": "{Key}",
                "expression": "grel:row.record.cells[\\"{Key}\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "{Value}",
                "columnInsertIndex": {i},
                "description": "Create column ``{Value}`` by filling up and down ``{Key}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Value}",
                "description": "Blank down cells in column ``{Value}``"
            }},
            {{
                "op": "core/column-removal",
                "columnName": "{Key}",
                "description": "Remove column ``{Key}``"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

## Organise ISBNs

**Organize_ISBN_Fields_1 NOT TESTED WITH RECORDS CONTAINING MULTIPLE ISBNS IN A SINGLE SUBFIELD, has length column but steps using it not checked**

Copy Organize_ISBN_Fields_1.json and paste into OpenRefine

Perform clustering on column "ISBN Type"

Copy below and paste into OpenRefine

`{`

`"op": "core/column-removal",`

`"columnName": "ISBN Subfield",`

`"description": "Remove column ISBN Subfield"`

`}`

In OpenRefine, All > Edit Columns > Fill Down

**No duplicates in sample, so need review on how deduping functions**

Copy Organize_ISBN_Fields_2.json and paste into OpenRefine

In OpenRefine, showing as rows, All > Edit Columns > Blank Down

In OpenRefine, undo steps blanking down cells in columns "ISBN" and "ISBN Type"

Open a text filter for "ISBN Type", click on the number of choices to get the facet choices as TSV, paste into variable "ISBN_Fields" below, and run cell

In [333]:
ISBN_Types = """
eISBN 13	524
ISBN 13	524
"""

Copy Organize_ISBN_Fields_3.json and paste into OpenRefine

In [334]:
ISBN_Types = ISBN_Types.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
ISBN_Fields = []
for ISBN_Type in ISBN_Types:
    ISBN_Type = ISBN_Type.rstrip("1234567890")
    ISBN_Type = ISBN_Type.rstrip("\t")
    ISBN_Fields.append(ISBN_Type)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for ISBN_Type in ISBN_Fields:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{ISBN_Type}",
                "expression": "grel:row.record.cells[\\"{ISBN_Type}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{ISBN_Type}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{ISBN_Type}",
                "description": "Blank down cells in column ``{ISBN_Type}``"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

# Organize HOL

> Start by loading Alpeh Sequential text file into OpenRefine with column widths 10,5,3

Put the unique name of the OpenRefine project for the HOL records in the variable `HOL_Project_Name` and run the cell

In [335]:
HOL_Project_Name = "oho_hol_3 18 2021 txt"

Copy Organize_Aleph_Sequential_for_HOL.json and paste into OpenRefine

Open a text filter for "Fields", click on the number of choices to get the facet choices as TSV, paste into variable `Number_of_TKRs` below, and run cell

In [336]:
Number_of_TKRs = """
TKR 1	524
"""

Copy below and paste into OpenRefine

`{`

`"op": "core/key-value-columnize",`

`"keyColumnName": "Fields",`

`"valueColumnName": "Values",`

`"noteColumnName": "",`

`"description": "Pivot data so values in ``Fields`` are columns with values from ``Values``"`

`}`

In [337]:
Number_of_TKRs = Number_of_TKRs.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
TKR_Fields = []
for TKR in Number_of_TKRs:
    TKR = TKR.rstrip("1234567890")
    TKR = TKR.rstrip("\t")
    TKR_Fields.append(TKR)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for TKR_Field in TKR_Fields:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{TKR_Field}",
                "expression": "grel:row.record.cells[\\"{TKR_Field}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{TKR_Field}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{TKR_Field}",
                "description": "Blank down cells in column ``{TKR_Field}``"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

**BEST WAY TO HANDLE MULTIPLE HOL FOR A BIB UNKNOWN: have column for number of HOL in BIB here, but don't know how to handle it if number is greater than one**

Copy Organize_TKRs.json and paste into OpenRefine